In [2]:
# setup cell
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent  # since notebooks/ is one level down
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import make_vec_env
import numpy as np

from uav_rl_navigation import config
from uav_rl_navigation.utils.env_helpers import make_flat_voyager
from uav_rl_navigation.environment.metrics_wrapper import UAVMetricsWrapper

from uav_rl_navigation.environment.vector_voyager_env import VectorVoyagerEnv
from uav_rl_navigation.vendor.pyflyt import FlattenVectorVoyagerEnv
from uav_rl_navigation.environment.cam_wrapper import ThirdPersonCamWrapper
from uav_rl_navigation.environment.metrics_wrapper import UAVMetricsWrapper

2025-06-20 10:43:57.948 | INFO     | uav_rl_navigation.config:<module>:11 - PROJ_ROOT path is: /Users/user/Documents/education/CAI/bachelor_thesis/uav-rl-navigation
pybullet build time: Mar 21 2025 17:10:00


In [3]:
# analyse environment characteristics
def analyse_env(env):
    # reset
    out = env.reset()
    if hasattr(env, "num_envs"):
        obs = out  # VecEnv only returns obs
        n = env.num_envs
        # build batch of actions
        actions = np.stack([env.action_space.sample() for _ in range(n)])
        batch = env.step(actions)
        # VecEnv.step returns (obs, rewards, dones, infos)
        obs2, rews, dones, infos = batch
        print(f"VecEnv:\n\nobs space:\n{env.observation_space}\n\nstacked obs sample:\n{obs}\n\naction space:\n{env.action_space}\n\nstacked actions sample:\n{actions}\n\nstacked reward samples:\n\n{rews}\n\nadditional info:\n\n{infos}\n")
    else:
        obs, info = out  # single env returns (obs, info)
        action = env.action_space.sample()
        obs2, rew, term, trunc, info2 = env.step(action)
        print(f"Single Env:\n\nobs space:\n{env.observation_space}\n\nobs sample:\n{obs}\n\naction space:\n{env.action_space}\n\naction sample:\n{action}\n\nreward sample:\n{rew}\n\nadditional infos:\n{info2}\n")

    # only vec environments have the env.envs attribute
    if hasattr(env, "envs"):
        inner_env = env.envs[0]  # grab the first inner env
    else:
        inner_env = env.env

    chain = []
    chain.append(type(env).__name__)  # add the name of the outer most env
    
    # now peel off the rest of the wrappers
    current = inner_env
    while True:
        # print(current)
        chain.append(type(current).__name__)
        # most wrappers keep the inner env in .env
        if not hasattr(current, "env"):
            break
        current = current.env

    print("Wrapper chain (outer -> inner):", " -> ".join(chain))

# init and check envs

In [39]:
# metrics env
env = make_flat_voyager(with_metrics=True)  # -> metrics env

# check env for Gymnasium and SB3 compatibility
check_env(env, warn=True)

# analyse basics 
analyse_env(env)

                             argv[0]=

                             
argv[0]=
                             
argv[0]=
                             
argv[0]=
Single Env:

obs space:
Box(-inf, inf, (13,), float32)

obs sample:
[ 0.         0.         0.         0.        -0.1444636  0.9912149
  0.         0.         0.         0.        -4.287521   3.9873655
 11.405745 ]

action space:
Box([-26.   -26.    -2.62  -8.  ], [26.   26.    2.62  8.  ], (4,), float32)

action sample:
[-2.2456248e+00  1.0381398e+01 -5.3184748e-01 -8.4062237e-03]

reward sample:
-0.06880740249707565

additional infos:
{'out_of_bounds': False, 'collision': False, 'env_complete': False, 'num_targets_reached': 0}

Wrapper chain (outer -> inner): UAVMetricsWrapper -> FlattenVectorVoyagerEnv -> VectorVoyagerEnv -> Aviary


/Users/user/Documents/education/CAI/bachelor_thesis/uav-rl-navigation/.venv/lib/python3.12/site-packages/stable_baselines3/common/env_checker.py:462: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


In [10]:
# instantiate vec_env with SB3
vec_env = make_vec_env(  # -> vec env
    make_flat_voyager, 
    n_envs=1, 
    env_kwargs=dict(num_waypoints=1, with_metrics=False), 
    monitor_dir= ".",
    monitor_kwargs=dict(),  # collect extra information to log, from the information return of env.step() - ep reward, ep length, ep time length  
) 

# analyse vec_env characteristics
analyse_env(vec_env)

vec_env

                             argv[0]=

VecEnv:

obs space:
Box(-inf, inf, (13,), float32)

stacked obs sample:
[[ 0.         0.         0.         0.        -0.1430722  0.991241
   0.         0.         0.         0.         9.667879   7.582403
   6.5568485]]

action space:
Box([-26.   -26.    -2.62  -8.  ], [26.   26.    2.62  8.  ], (4,), float32)

stacked actions sample:
[[-17.891075    25.44432     -0.18201828  -0.28371212]]

stacked reward samples:

[-0.07128291]

additional info:

[{'out_of_bounds': False, 'collision': False, 'env_complete': False, 'num_targets_reached': 0, 'TimeLimit.truncated': False}]

Wrapper chain (outer -> inner): DummyVecEnv -> Monitor -> FlattenVectorVoyagerEnv -> VectorVoyagerEnv -> Aviary


In [ ]:
# explore envs
base_env = VectorVoyagerEnv()
obs, info = base_env.reset()
# analyse_env(base_env)
base_env.unwrapped
aviary = base_env.env
aviary.env

                             argv[0]=



AttributeError: module 'pybullet' has no attribute 'env'